In [17]:
import numpy as np
import pandas as pd
from keras.models import load_model
import glob
import joblib

In [10]:
TRAIN_PATH = '../data/final_model/train/train_data.joblib'
FOLDER_MODEL = '../model/FINAL_MODEL/03-06-2024_11-51/'

In [11]:
def load_data(path_data: str) -> tuple:
    X = None
    y = None
    data_loaded = joblib.load(path_data)
    X = data_loaded['X']
    y = data_loaded['y']
    
    all_stars = np.array([s.split('_')[1] for s in data_loaded['sample_names']]) # lấy ra star alen: 10, 1, 2, ...
    stars, idx = np.unique(all_stars, return_index=True)
    sample_mask = np.isin(all_stars, all_stars[idx]) # Đánh dấu những star allele của idx trong all_stars
    stars_001 = np.array([s for s in data_loaded['sample_names'][sample_mask] if s.split('_')[-1] == '001']) 
    mark_001 = np.isin(data_loaded['sample_names'], stars_001)
    X_001, y_001 = X[mark_001], y[mark_001]
    
    dataset = [(X, y), (X_001, y_001)]
    
    return dataset

In [12]:
dataset_trainning = load_data(path_data=TRAIN_PATH)

In [18]:
X = dataset_trainning[0][0]
y = dataset_trainning[0][1]

In [14]:
def predict_probs(X):
    models = glob.glob(FOLDER_MODEL + "*.model.h5")
    predicts = [] 
    for m in models: 
        model = load_model(m)       
        predict = model.predict(X)
        predicts.append(predict)
        
    return np.array(predicts).mean(axis=0)

In [35]:
def save2csv(probs, y):
    match = np.concatenate((y, probs), axis=1)
    df = pd.DataFrame(match, columns=['origin_label_1', 'origin_label_2', 'No.Function', 'Normal.Function'])
    df['origin_label_1'] = df['origin_label_1'].apply(lambda x: 'Y' if x == 1.0 else 'N')
    df['origin_label_2'] = df['origin_label_2'].apply(lambda x: 'Y' if x == 1.0 else 'N')
    
    df.to_csv('../data/final_model/probs_getThreshold.csv', index=False)
    print('Done!')

In [27]:
probs = predict_probs(X)

5/5 [==============================] - 0s 32ms/step


In [36]:
save2csv(probs, y)

Done!
